# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

In [1]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)


Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:utils':

    timestamp



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"пакет 'plotly' был собран под R версии 4.4.2"

Присоеди

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [5]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$cases_i = df$cases
df$cases_j = df$`cases:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,C1M_School_closing_i,C1M_School_closing_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,1.183562,1.49315071,248.620682,4.055137,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,1.290411,0.07671233,8.838488,89.445110,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,1.584700,1.98087430,26.542530,531.109006,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,2.153005,2.40437150,108.343854,98.596161,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,1.00000000,179.857692,63.377090,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,2.000000,0.27945206,122.707272,1660.971875,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [6]:
# Делаю pulled OLS ради интереса
m0 = feols(log(v/v19) ~ log(cases_i) + log(cases_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df)
m0 %>% summary(cluster = ~ i+j)

NOTE: 11,993 observations removed because of NA and infinite values (RHS: 11,993).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 58,314
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error  t value   Pr(>|t|)
(Intercept)                           0.227290   0.032187  7.06154 3.7831e-11
log(cases_i)                          0.007089   0.003753  1.88904 6.0551e-02
log(cases_j)                          0.005438   0.003505  1.55171 1.2255e-01
C8EV_International_travel_controls_i -0.041343   0.010857 -3.80786 1.9408e-04
C8EV_International_travel_controls_j -0.066083   0.010724 -6.16187 4.8524e-09
                                        
(Intercept)                          ***
log(cases_i)                         .  
log(cases_j)                            
C8EV_International_travel_controls_i ***
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.00224   Adj. R2: 0.009596

In [7]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
m1 = feols(log(v/v19) ~ log(cases_i) + log(cases_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year,
 data = df)
m1 %>% summary(cluster = ~ i+j)

NOTE: 11,993 observations removed because of NA and infinite values (RHS: 11,993).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 58,314
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error  t value  Pr(>|t|)
log(cases_i)                         -0.016700   0.005789 -2.88505 0.0044087
log(cases_j)                          0.004826   0.004582  1.05326 0.2936840
C8EV_International_travel_controls_i -0.021656   0.013379 -1.61861 0.1073427
C8EV_International_travel_controls_j -0.018229   0.013345 -1.36601 0.1736999
                                        
log(cases_i)                         ** 
log(cases_j)                            
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.984614     Adj. R2: 0.038351
                 Within R2: 3.983e-4

In [8]:
# Добавляю гетерогенность эффектов во времени
m2 = feols(log(v/v19) ~ year/log(cases_i) + year/log(cases_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
m2 %>% summary(cluster = ~ i+j)

NOTE: 11,993 observations removed because of NA and infinite values (RHS: 11,993).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 58,314
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                       0.025591   0.084314  0.303517
year2022                                       0.097048   0.099576  0.974615
year2020:log(cases_i)                         -0.020013   0.007134 -2.805360
year2021:log(cases_i)                         -0.015182   0.007536 -2.014455
year2022:log(cases_i)                         -0.016445   0.006029 -2.727707
year2020:log(cases_j)                         -0.001258   0.005859 -0.214774
year2021:log(cases_j)                          0.008710   0.007932  1.098207
year2022:log(cases_j)                          0.009976   0.005772  1.728480
year2020:C8EV_International_travel_controls_i -0.051941   0.024711 -2.101954
year2021:C8EV_International_travel_controls_i -0.025328   0.012859 -1.969648
year2022:C8EV_International

In [9]:
# С кластеризацией ошибок есть значимый эффект и наблюдается гетерогенность во времени (+ направление связи соотносится с описанным в литературе)
etable(m0, m1, m2, vcov = ~i+j, signif.code=signif.code)
# А чем такая кластеризация отличается от i^j?

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2273*** (0.0322),,
4,log(cases_i),0.0071. (0.0038),-0.0167** (0.0058),
5,log(cases_j),0.0054 (0.0035),0.0048 (0.0046),
6,C8EV_International_travel_controls_i,-0.0413*** (0.0109),-0.0217 (0.0134),
7,C8EV_International_travel_controls_j,-0.0661*** (0.0107),-0.0182 (0.0133),
8,year2021,,,0.0256 (0.0843)
9,year2022,,,0.0970 (0.0996)


In [10]:
# При использовании НАС ошибок все эффекты становятся незначимыми
etable(m0, m1, m2, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2273* (0.0231),,
4,log(cases_i),0.0071 (0.0027),-0.0167. (0.0051),
5,log(cases_j),0.0054 (0.0025),0.0048 (0.0048),
6,C8EV_International_travel_controls_i,-0.0413* (0.0071),-0.0217 (0.0120),
7,C8EV_International_travel_controls_j,-0.0661* (0.0072),-0.0182 (0.0118),
8,year2021,,,0.0256 (0.0882)
9,year2022,,,0.0970 (0.0892)


In [11]:
# Всё-таки кластеризовать только по году нельзя, потому что мы в формуле указываем кусочки ошибок которые скоррелированные
    # Пробую кластеризацию ошибок по годам: с теоретической точки зрения это обосновано тем,
    # что все связанные с одной страной наблюдения связаны, значит и во всём графе наблюдения связаны.
    # При этом если нет автокорреляции (учитывая, что у меня 3 периода, это вроде ОК), то кластеризация по годам именно так и делает.
    # Но в статьях пишут, что нельзя использовать кластеризацию ошибок, если меньше чем 50 кластеров...
    # Интересно, что здесь эффект импортёра значимый, а в предыдущей спецификации эффект экспортёра...
etable(m0, m1, m2, vcov = ~i+j+year, signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2273 (0.0800),,
4,log(cases_i),0.0071 (0.0069),-0.0167. (0.0057),
5,log(cases_j),0.0054 (0.0039),0.0048 (0.0038),
6,C8EV_International_travel_controls_i,-0.0413 (0.0169),-0.0217 (0.0114),
7,C8EV_International_travel_controls_j,-0.0661 (0.0301),-0.0182 (0.0118),
8,year2021,,,0.0256 (0.0690)
9,year2022,,,0.0970 (0.0907)


In [12]:
# Не понятно, насколько это осмысленно, но сделал на всякий случай
etable(m0, m1, m2, vcov = ~i^year+j^year, signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.2273*** (0.0329),,
4,log(cases_i),0.0071* (0.0033),-0.0167*** (0.0042),
5,log(cases_j),0.0054. (0.0030),0.0048 (0.0042),
6,C8EV_International_travel_controls_i,-0.0413*** (0.0114),-0.0217* (0.0103),
7,C8EV_International_travel_controls_j,-0.0661*** (0.0112),-0.0182. (0.0110),
8,year2021,,,0.0256 (0.0678)
9,year2022,,,0.0970 (0.0776)


### Инструментальные переменные на панельных данных

In [14]:
# Pulled TSLS
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
m3 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(cases_i) + log(cases_j) 
 ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
population_density + population_density_j
  ,
 data = df
 )
m3 %>% summary(vcov = ~i+j)

NOTE: 12,332 observations removed because of NA and infinite values (RHS: 9,802, IV: 3,268/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(cases_i), log(cases_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 57,975
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                          -0.261155   0.188837 -1.382964 0.1684480
fit_log(cases_i)                      0.042156   0.036011  1.170643 0.2433430
fit_log(cases_j)                      0.072869   0.027493  2.650405 0.0087809
C8EV_International_travel_controls_i -0.014487   0.017726 -0.817287 0.4148820
C8EV_International_travel_controls_j -0.032792   0.016984 -1.930697 0.0551462
                                        
(Intercept)                             
fit_log(cases_i)                        
fit_log(cases_j)                     ** 
C8EV_International_travel_controls_i    
C8EV_International_t

In [15]:
# Только FE года (инструменты эндогенные)
m4 = feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year|
 log(cases_i) + log(cases_j) ~ 
 median_age_i + median_age_j 
+
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
  population_density + population_density_j
 ,
 data = df
 )
m4 %>% summary(cluster = ~ i+j)

NOTE: 12,332 observations removed because of NA and infinite values (RHS: 9,802, IV: 3,268/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(cases_i), log(cases_j)
                  Instr.   : median_age_i, median_age_j, H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 57,975
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(cases_i)                     -0.002855   0.006341 -0.450210 0.653120
fit_log(cases_j)                     -0.006925   0.005469 -1.266198 0.207135
C8EV_International_travel_controls_i -0.001089   0.013504 -0.080609 0.935846
C8EV_International_travel_controls_j -0.026780   0.013011 -2.058198 0.041062
                                        
fit_log(cases_i)                        
fit_log(cases_j)                        
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05

In [16]:
# Маски эндогенные :(
feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(cases_i) + log(cases_j) ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 12,332 observations removed because of NA and infinite values (RHS: 9,802, IV: 3,268/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(cases_i), log(cases_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 57,975
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(cases_i)                     -0.043361   0.019808 -2.189030 0.029926
fit_log(cases_j)                      0.010490   0.016187  0.648041 0.517813
C8EV_International_travel_controls_i -0.028496   0.015325 -1.859415 0.064657
C8EV_International_travel_controls_j -0.013690   0.013010 -1.052200 0.294167
                                        
fit_log(cases_i)                     *  
fit_log(cases_j)                        
C8EV_International_travel_controls_i .  
C8EV_International_travel_con

In [17]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 
# Добавить больше инструментов, которые меняются во времени. Мб плотность населения или медианный возраст брать не за 2019 год а в динамике, 
# чтобы использовать их как инструменты в этой спецификации.
m5 = feols(log(v/v19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(cases_i) + log(cases_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m5 %>% summary(cluster = ~ i+j)

NOTE: 12,332 observations removed because of NA and infinite values (RHS: 9,802, IV: 3,268/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(cases_i), log(cases_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 57,975
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(cases_i)                     -0.046168   0.020240 -2.280965 0.023763
fit_log(cases_j)                      0.021101   0.017210  1.226072 0.221828
C8EV_International_travel_controls_i -0.029463   0.015718 -1.874446 0.062545
C8EV_International_travel_controls_j -0.010579   0.013206 -0.801074 0.424181
                                        
fit_log(cases_i)                     *  
fit_log(cases_j)                        
C8EV_International_travel_controls_i .  
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '*

In [18]:
etable(m3, m4, m5, vcov = ~i+j, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,-0.2612 (0.1888),,
4,log(cases_i),0.0422 (0.0360),-0.0029 (0.0063),-0.0462* (0.0202)
5,log(cases_j),0.0729** (0.0275),-0.0069 (0.0055),0.0211 (0.0172)
6,C8EV_International_travel_controls_i,-0.0145 (0.0177),-0.0011 (0.0135),-0.0295. (0.0157)
7,C8EV_International_travel_controls_j,-0.0328. (0.0170),-0.0268* (0.0130),-0.0106 (0.0132)
8,Fixed-Effects:,-----------------,-----------------,-----------------
9,year,No,Yes,Yes


In [19]:
etable(m3, m4, m5, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,-0.2612. (0.0836),,
4,log(cases_i),0.0422 (0.0147),-0.0029 (0.0042),-0.0462. (0.0157)
5,log(cases_j),0.0729* (0.0143),-0.0069 (0.0037),0.0211 (0.0144)
6,C8EV_International_travel_controls_i,-0.0145 (0.0089),-0.0011 (0.0094),-0.0295 (0.0131)
7,C8EV_International_travel_controls_j,-0.0328. (0.0090),-0.0268 (0.0092),-0.0106 (0.0124)
8,Fixed-Effects:,-----------------,----------------,-----------------
9,year,No,Yes,Yes


In [20]:
    # Всё ещё странно использовать кластеризацию по году... 
    # etable(m3, m4, vcov = function(x) vcovHC(x, method="arellano", type="HC1"), signif.code = signif.code) Не сработало
etable(m3, m4, m5, vcov = ~i+j+year, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,-0.2612 (0.2188),,
4,log(cases_i),0.0422 (0.0353),-0.0029 (0.0072),-0.0462. (0.0150)
5,log(cases_j),0.0729 (0.0303),-0.0069 (0.0058),0.0211 (0.0176)
6,C8EV_International_travel_controls_i,-0.0145 (0.0163),-0.0011 (0.0104),-0.0295 (0.0162)
7,C8EV_International_travel_controls_j,-0.0328 (0.0126),-0.0268 (0.0212),-0.0106 (0.0128)
8,Fixed-Effects:,----------------,----------------,-----------------
9,year,No,Yes,Yes


In [21]:
# Так вроде не очень корректно делать, если мы верим в возможную автокорреляцию
etable(m3, m4, m5, vcov = ~i^year+j^year, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,-0.2612 (0.1700),,
4,log(cases_i),0.0422 (0.0277),-0.0029 (0.0045),-0.0462** (0.0163)
5,log(cases_j),0.0729** (0.0228),-0.0069. (0.0040),0.0211 (0.0144)
6,C8EV_International_travel_controls_i,-0.0145 (0.0178),-0.0011 (0.0122),-0.0295* (0.0127)
7,C8EV_International_travel_controls_j,-0.0328* (0.0157),-0.0268* (0.0117),-0.0106 (0.0114)
8,Fixed-Effects:,-----------------,-----------------,------------------
9,year,No,Yes,Yes


In [ ]:
# С помощью fixest::demean можно отрисовать scatterplot, который обосновывает необходимость использовать фиксированные эффекты
# и релевантность инструментов

## Три пространственные регрессии (как в курсовой)

Но в такую спецификацию нельзя добавить фиксированные эффекты, поэтому почти наверное без инструментов будет смещение из-за пропущенных переменных

In [22]:
# Без инструментов.
m_2020 = feols(log(v/v19) ~  1+log(cases_i) + log(cases_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v/v19) ~  1+log(cases_i) + log(cases_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v/v19) ~  1+log(cases_i) + log(cases_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)

NOTE: 4,318 observations removed because of NA and infinite values (RHS: 4,318).

NOTE: 3,960 observations removed because of NA and infinite values (RHS: 3,960).

NOTE: 3,715 observations removed because of NA and infinite values (RHS: 3,715).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 19,604
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error  t value  Pr(>|t|)
(Intercept)                           0.062281   0.049065  1.26934 0.2043349
log(cases_i)                         -0.003662   0.003026 -1.21020 0.2262181
log(cases_j)                         -0.007656   0.002878 -2.65995 0.0078215
C8EV_International_travel_controls_i -0.016091   0.012979 -1.23975 0.2150818
C8EV_International_travel_controls_j -0.033410   0.012406 -2.69314 0.0070842
                                        
(Intercept)                             
log(cases_i)                            
log(cases_j)                         ** 
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.883335   Adj. R2: 6.845e-4

OLS estimation, Dep. Var.: log(v/v19)
Observations: 19,598
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           0.132235   0.044360  2.980926 0.0028773
log(cases_i)                          0.003422   0.004537  0.754200 0.4507383
log(cases_j)                         -0.004860   0.003983 -1.220178 0.2224120
C8EV_International_travel_controls_i -0.005372   0.009862 -0.544654 0.5859977
C8EV_International_travel_controls_j -0.004172   0.010079 -0.413939 0.6789230
                                        
(Intercept)                          ** 
log(cases_i)                            
log(cases_j)                            
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.0267   Adj. R2: -6.56e-5

OLS estimation, Dep. Var.: log(v/v19)
Observations: 19,112
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.254047   0.038735  6.558660 5.5684e-11
log(cases_i)                         -0.001957   0.004017 -0.487148 6.2616e-01
log(cases_j)                         -0.000883   0.003547 -0.248951 8.0340e-01
C8EV_International_travel_controls_i  0.026567   0.018822  1.411486 1.5812e-01
C8EV_International_travel_controls_j -0.062353   0.016940 -3.680874 2.3307e-04
                                        
(Intercept)                          ***
log(cases_i)                            
log(cases_j)                            
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.07887   Adj. R2: 7.065e-4

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0623 (0.0491),0.1322** (0.0444),0.2540*** (0.0387)
4,log(cases_i),-0.0037 (0.0030),0.0034 (0.0045),-0.0020 (0.0040)
5,log(cases_j),-0.0077** (0.0029),-0.0049 (0.0040),-0.0009 (0.0035)
6,C8EV_International_travel_controls_i,-0.0161 (0.0130),-0.0054 (0.0099),0.0266 (0.0188)
7,C8EV_International_travel_controls_j,-0.0334** (0.0124),-0.0042 (0.0101),-0.0624*** (0.0169)
8,____________________________________,__________________,_________________,___________________
9,S.E. type,Heteroskedas.-rob.,Heteroskeda.-rob.,Heteroskedast.-rob.


In [23]:
# С инструментами оценки коэффициентов всё равно не похожи на панельные данные (пространственные смещённые, т.к должно быть много unobservables)
# Использую ошибки hc1
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
# Почему экзогенные в панельной спецификации инструменты эндогенны в пространственной?
m_2020 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(cases_i) + log(cases_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(cases_i) + log(cases_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(v/v19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(cases_i) + log(cases_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)
etable(m_2020, m_2021, m_2022, vcov = ~i+j, signif.code = signif.code)

NOTE: 4,318 observations removed because of NA and infinite values (RHS: 3,419, IV: 1,429/3,419).

NOTE: 3,960 observations removed because of NA and infinite values (RHS: 3,290, IV: 1,124/3,290).

NOTE: 4,054 observations removed because of NA and infinite values (RHS: 3,093, IV: 715/3,952).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(cases_i), log(cases_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 19,604
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           0.059902   0.053191  1.126171 0.2601071
fit_log(cases_i)                      0.003360   0.005141  0.653519 0.5134294
fit_log(cases_j)                     -0.013851   0.004690 -2.953586 0.0031448
C8EV_International_travel_controls_i -0.016112   0.012997 -1.239589 0.2151425
C8EV_International_travel_controls_j -0.033579   0.012408 -2.706217 0.0068113
                                        
(Intercept)                             
fit_log(cases_i)                        
fit_log(cases_j)                     ** 
C8EV_International_travel_c

TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(cases_i), log(cases_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 19,598
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value  Pr(>|t|)
(Intercept)                           0.139106   0.053727  2.589138 0.0096288
fit_log(cases_i)                      0.001401   0.006670  0.210110 0.8335844
fit_log(cases_j)                     -0.004633   0.005751 -0.805527 0.4205252
C8EV_International_travel_controls_i -0.004950   0.009853 -0.502396 0.6153944
C8EV_International_travel_controls_j -0.004325   0.010106 -0.427935 0.6687029
                                        
(Intercept)                          ** 
fit_log(cases_i)                        
fit_log(cases_j)                        
C8EV_International_travel_c

TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(cases_i), log(cases_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 18,773
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                           0.334638   0.043356  7.718417 1.2367e-14
fit_log(cases_i)                     -0.009722   0.004525 -2.148435 3.1692e-02
fit_log(cases_j)                     -0.007260   0.004134 -1.756230 7.9066e-02
C8EV_International_travel_controls_i  0.016110   0.019130  0.842110 3.9974e-01
C8EV_International_travel_controls_j -0.077534   0.017548 -4.418271 1.0005e-05
                                        
(Intercept)                          ***
fit_log(cases_i)                     *  
fit_log(cases_j)                     .  
C8EV_International_tr

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0599 (0.0532),0.1391** (0.0537),0.3346*** (0.0434)
4,log(cases_i),0.0034 (0.0051),0.0014 (0.0067),-0.0097* (0.0045)
5,log(cases_j),-0.0138** (0.0047),-0.0046 (0.0058),-0.0073. (0.0041)
6,C8EV_International_travel_controls_i,-0.0161 (0.0130),-0.0050 (0.0099),0.0161 (0.0191)
7,C8EV_International_travel_controls_j,-0.0336** (0.0124),-0.0043 (0.0101),-0.0775*** (0.0175)
8,____________________________________,__________________,_________________,___________________
9,S.E. type,Heteroskedas.-rob.,Heteroskeda.-rob.,Heteroskedast.-rob.


,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,Constant,0.0599 (0.0643),0.1391. (0.0767),0.3346*** (0.0870)
4,log(cases_i),0.0034 (0.0060),0.0014 (0.0093),-0.0097 (0.0080)
5,log(cases_j),-0.0138* (0.0063),-0.0046 (0.0080),-0.0073 (0.0068)
6,C8EV_International_travel_controls_i,-0.0161 (0.0176),-0.0050 (0.0134),0.0161 (0.0451)
7,C8EV_International_travel_controls_j,-0.0336. (0.0180),-0.0043 (0.0134),-0.0775. (0.0420)
8,____________________________________,_________________,________________,__________________
9,S.E.: Clustered,by: i & j,by: i & j,by: i & j
